In [2]:
from chess_env import ChessEnv
from network import Agent

import torch

# https://www.kaggle.com/code/arjanso/reinforcement-learning-chess-3-q-networks

env = ChessEnv()
agent = Agent(
    gamma=0.99,
    epsilon=1.0,
    batch_size=128,
    max_me_size=1_000,
    eps_end=0.01,
    input_dims=[390, 390],
    lr=0.003
)
scores = []
games = {}
number_of_games = 20

agent.Q_eval.device
agent.Q_eval.module_list[0].forward(torch.tensor(env.read_board(), dtype=torch.float))

RuntimeError: Given groups=1, weight of size [200, 200, 3, 3], expected input[1, 6, 8, 8] to have 200 channels, but got 6 channels instead

In [ ]:
for game in range(number_of_games):
    score = 0
    observation, info = env.reset()
    while not env.game.is_game_over():
        current_action_space = env.action_space
        action = agent.choose_action(observation, current_action_space)
        new_observation, reward, ended, _, info = env.step(action)
        score += reward
        agent.store_transition(
            observation,
            current_action_space.index(action),
            reward,
            new_observation,
            ended
        )
        agent.learn(current_action_space, env.action_space)
        observation = new_observation

    games[env.pgn()] = {
        "scores" : scores,
        "eps_hist" : agent.epsilon
    }

    print(f"Episode: {game}. Score: {score:.2f}. Epsilon: {agent.epsilon:.2f}.")

ValueError: could not broadcast input array from shape (6,8,8) into shape (390,390)